In [20]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from matplotlib import pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import sklearn.metrics as skm
import pickle

In [22]:
test_data = pd.read_csv("E:/jigsaw-toxic-comment-classification-challenge/test.csv/test.csv")
test_data

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [23]:
P = test_data.iloc[0:10000]
P

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
9995,10b1754d1898042c,Justin Timberlake is Gay
9996,10b200880206ed38,the blokes a welsh twat
9997,10b23e9152acee30,== A belated welcome! == \n\n ]]Here's wishing...
9998,10b2810192fb1174,""", 26 September 2010 (UTC) \n :::::::Ugghhhh e..."


In [24]:
P = P["comment_text"]
P

0       Yo bitch Ja Rule is more succesful then you'll...
1       == From RfC == \n\n The title is fine as it is...
2       " \n\n == Sources == \n\n * Zawe Ashton on Lap...
3       :If you have a look back at the source, the in...
4               I don't anonymously edit articles at all.
                              ...                        
9995                             Justin Timberlake is Gay
9996                              the blokes a welsh twat
9997    == A belated welcome! == \n\n ]]Here's wishing...
9998    ", 26 September 2010 (UTC) \n :::::::Ugghhhh e...
9999                      ::Hooray! A non-POV discussion!
Name: comment_text, Length: 10000, dtype: object

In [25]:
P = P.str.lower()
P

0       yo bitch ja rule is more succesful then you'll...
1       == from rfc == \n\n the title is fine as it is...
2       " \n\n == sources == \n\n * zawe ashton on lap...
3       :if you have a look back at the source, the in...
4               i don't anonymously edit articles at all.
                              ...                        
9995                             justin timberlake is gay
9996                              the blokes a welsh twat
9997    == a belated welcome! == \n\n ]]here's wishing...
9998    ", 26 september 2010 (utc) \n :::::::ugghhhh e...
9999                      ::hooray! a non-pov discussion!
Name: comment_text, Length: 10000, dtype: object

In [26]:
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
P = P.apply(lambda x:decontracted(x))
P

0       yo bitch ja rule is more succesful then you wi...
1       == from rfc == \n\n the title is fine as it is...
2       " \n\n == sources == \n\n * zawe ashton on lap...
3       :if you have a look back at the source, the in...
4              i do not anonymously edit articles at all.
                              ...                        
9995                             justin timberlake is gay
9996                              the blokes a welsh twat
9997    == a belated welcome! == \n\n ]]here is wishin...
9998    ", 26 september 2010 (utc) \n :::::::ugghhhh e...
9999                      ::hooray! a non-pov discussion!
Name: comment_text, Length: 10000, dtype: object

In [27]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves', 'article', 'please', 'talk', 'wikipedia', 'one', 'utc', 'see', 'user', 'redirect', 'question', 'image'
                'stop', 'need', 'seem', 'welcome', 'link', 'use', 'page', 'thank', 'made', 'hello', 'know', 'back', 'much',
               'right', 'section', 'people', 'hi', 'may', 'said', 'list', 'well', 'read', 'u', 'agree', 'help', 'really',
               'name', 'deletion', 'hey', 'say', 'someone', 'want', 'look', 'edit', 'make', 'comment', 'source', 'think',
               'time', 'still', 'editing', 'sure', 'delete', 'work', 'july', 'add', 'okay', 'dont', 'way', 'another']
STOPWORDS = set(stopwordlist)
def cleaning_stopwords(P):
    P = " ".join([word for word in str(P).split() if word not in STOPWORDS])
    return P
P = P.apply(lambda x:cleaning_stopwords(x))
P

0       yo bitch ja rule succesful ever whats hating s...
1                           == rfc == title fine is, imo.
2             " == sources == * zawe ashton lapland — / "
3       :if source, information updated correct form. ...
4                           not anonymously articles all.
                              ...                        
9995                                justin timberlake gay
9996                                    blokes welsh twat
9997    == belated welcome! == ]]here wishing belated ...
9998    ", 26 september 2010 (utc) :::::::ugghhhh emai...
9999                        ::hooray! non-pov discussion!
Name: comment_text, Length: 10000, dtype: object

In [28]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(P):
    translator = str.maketrans('', '', punctuations_list)
    return P.translate(translator)
P = P.apply(lambda x: cleaning_punctuations(x))
P

0       yo bitch ja rule succesful ever whats hating s...
1                                  rfc  title fine is imo
2                       sources   zawe ashton lapland —  
3       if source information updated correct form gue...
4                            not anonymously articles all
                              ...                        
9995                                justin timberlake gay
9996                                    blokes welsh twat
9997     belated welcome  here wishing belated wikiped...
9998     26 september 2010 utc ugghhhh email client go...
9999                             hooray nonpov discussion
Name: comment_text, Length: 10000, dtype: object

In [29]:
def cleaning_URLs(P):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',P)
P = P.apply(lambda x: cleaning_URLs(x))
P

0       yo bitch ja rule succesful ever whats hating s...
1                                  rfc  title fine is imo
2                       sources   zawe ashton lapland —  
3       if source information updated correct form gue...
4                            not anonymously articles all
                              ...                        
9995                                justin timberlake gay
9996                                    blokes welsh twat
9997     belated welcome  here wishing belated wikiped...
9998     26 september 2010 utc ugghhhh email client go...
9999                             hooray nonpov discussion
Name: comment_text, Length: 10000, dtype: object

In [30]:
def cleaning_numbers(P):
    return re.sub('[0-9]+', '', P)
P = P.apply(lambda x: cleaning_numbers(x))
P

0       yo bitch ja rule succesful ever whats hating s...
1                                  rfc  title fine is imo
2                       sources   zawe ashton lapland —  
3       if source information updated correct form gue...
4                            not anonymously articles all
                              ...                        
9995                                justin timberlake gay
9996                                    blokes welsh twat
9997     belated welcome  here wishing belated wikiped...
9998      september  utc ugghhhh email client gone scr...
9999                             hooray nonpov discussion
Name: comment_text, Length: 10000, dtype: object

In [31]:
tokenizer = RegexpTokenizer('\s+', gaps=True)
P = P.apply(tokenizer.tokenize)
P

0       [yo, bitch, ja, rule, succesful, ever, whats, ...
1                             [rfc, title, fine, is, imo]
2                     [sources, zawe, ashton, lapland, —]
3       [if, source, information, updated, correct, fo...
4                       [not, anonymously, articles, all]
                              ...                        
9995                            [justin, timberlake, gay]
9996                                [blokes, welsh, twat]
9997    [belated, welcome, here, wishing, belated, wik...
9998    [september, utc, ugghhhh, email, client, gone,...
9999                         [hooray, nonpov, discussion]
Name: comment_text, Length: 10000, dtype: object

In [32]:
lm = WordNetLemmatizer()
def lemmatization(P):
    P = [lm.lemmatize(word) for word in P]
    return P
P = P.apply(lambda x: lemmatization(x))
P

0       [yo, bitch, ja, rule, succesful, ever, whats, ...
1                             [rfc, title, fine, is, imo]
2                      [source, zawe, ashton, lapland, —]
3       [if, source, information, updated, correct, fo...
4                        [not, anonymously, article, all]
                              ...                        
9995                            [justin, timberlake, gay]
9996                                 [bloke, welsh, twat]
9997    [belated, welcome, here, wishing, belated, wik...
9998    [september, utc, ugghhhh, email, client, gone,...
9999                         [hooray, nonpov, discussion]
Name: comment_text, Length: 10000, dtype: object

In [33]:
def list_to_string(P):
    P = (' '.join(map(str, P)))
    return P
P = P.apply(lambda x: list_to_string(x))
P

0       yo bitch ja rule succesful ever whats hating s...
1                                   rfc title fine is imo
2                            source zawe ashton lapland —
3       if source information updated correct form gue...
4                             not anonymously article all
                              ...                        
9995                                justin timberlake gay
9996                                     bloke welsh twat
9997    belated welcome here wishing belated wikipedia...
9998    september utc ugghhhh email client gone screwy...
9999                             hooray nonpov discussion
Name: comment_text, Length: 10000, dtype: object

In [34]:
vectorizer = pickle.load(open("vectorizer.pickle", "rb"))

In [35]:
P=vectorizer.transform(P)

In [36]:
P.shape

(10000, 8000)

In [37]:
pickled_model = pickle.load(open('model.pkl', 'rb'))
Q=pd.DataFrame(pickled_model.predict_proba(P))
Q.insert(0,'id',test_data.id,True)
Q.columns=['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']
Q

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999994,0.080616,0.999982,0.007398,0.875384,0.050554
1,0000247867823ef7,0.033413,0.005502,0.011553,0.002161,0.012181,0.007286
2,00013b17ad220c46,0.003727,0.004363,0.003925,0.001948,0.009619,0.004566
3,00017563c3f7919a,0.006338,0.003902,0.006702,0.002224,0.011188,0.005862
4,00017695ad8997eb,0.002918,0.003959,0.009937,0.001728,0.014097,0.007678
...,...,...,...,...,...,...,...
9995,10b1754d1898042c,1.000000,0.008762,0.020177,0.003325,0.102837,0.824018
9996,10b200880206ed38,0.859536,0.005482,0.026126,0.002097,0.037265,0.011358
9997,10b23e9152acee30,0.022246,0.004138,0.009352,0.002265,0.014721,0.004722
9998,10b2810192fb1174,0.065124,0.004715,0.014880,0.002342,0.025423,0.009118


In [39]:
Q.to_excel("E:/jigsaw-toxic-comment-classification-challenge/output_file.xlsx",index= False)